#### title: "Strategies for Analyzing a 12-Gigabyte Data Set: Airline Flight Delays - Case Study Unit 14

output: html_notebook


#### Cory Adams, Chris Boomhower, Alexandra Fisher, Alex Frye
#### MSDS 7333, December 10, 2017

## Abstract
This case study was undertaken in order to investigate and highlight how to work with datasets that are too large in size to fit into local memory. To do this, 12 gigabytes of historical flight data for on-time performance is selected and the data is loaded and analyzed out-of-core using a branch processing method [2]. For purposes of this study, the split-apply-combine approach is used and both of the R and Python languages are leveraged to manage the data out-of-core. Results are then used to ultimately determine if consumers can reduce the number of flight delays experienced by answering key questions, such as which airports and flights are most likely to be delayed. Findings indicate the [insert airports] are the airports most likely to be delayed flying out of or into and the flights with [insert origin] as the origin and [insert destination] as the destination are the most likely to be delayed. 

Can we regress how delayed a flight will be before it is delayed? 
If yes, how? And what are the most important features for this regression? 


## Introduction
In this case study, the objective is not only to investigate how to work with oversized data sets too massive for local memory, but to answer key questions pertaining to airline flight performance. Specifically, a key measurement of success in the airline industry is on-time performance, which is an indicator of flight delays. The following questions of interest will be answered in this study:

1.	Which airports are most likely to be delayed flying out of or into?
2.	Which flights with same origin and destination are most likely to be delayed? 
3.	Can we regress how delayed a flight will be before it is delayed? What are the most important features for this regression? 

In answering these key questions, this study will aid in determining if consumers can reduce the quantity of flight delays experienced via analyzing historical flight data collected between 1987 and 2009 [2]. The airline data set used can be found at http://stat-computing.org/dataexpo/2009/the-data.html.

To best handle loading and analyzing the large airline dataset, the data must be processed out-of-core via a branch processing method. In particular, the split-apply-combine technique is utilized with various APIs. For the purpose of this study, R and Python languages are leveraged to manage the data out-of-core. 


## Background
The “Airline on-time performance” data set [2] comes from the American Statistical Association (ASA) Section on Statistical Computing and Statistical Graphics released in 2009 “for their biannual data exposition” [1]. The data set was compiled and released from the U.S. Government’s Bureau of Transportation Research and Innovative Technology Administration (RITA) and includes flight information from October 1987 to April 2008 for over 120 million flights. Flight data includes a total of 29 variables for each flight, such as flight time, delay time, departure airport, and arrival airport, equaling a total of 12 gigabytes in files. 

In particular, a researcher from Yale University named Michael Kane strategizes how to analyze massive datasets using the “Airline on-time performance” data set. Kane provides a case study that explains how to acquire the data set, explore the data set using different environments such as R, Unix, and SQL db and parallel computing, and model the airline data set [3 & 4]. 

## Methods
The steps used for this analysis were: 1) data acquisition and decompression, 2) loading data into memory using both R and Python to verify decompression, 3) preprocessing data in chunks, and 4) data analysis out-of-core using Python. 

Note that code used includes modified versions of R code function examples found in Data Science in R: A Case Studies Approach to Computational Reasoning and Problem Solving, Chapter 5, pages 217-236 [1].

## Results

# Batch Processing in R and Python
This is meant as a companion notebook for the the SMU MSDS program. Specifically, we will build off of chapter five in the text "Data Science in R", http://www.amazon.com/Data-Science-Approach-Computational-Reasoning/dp/1482234815.

In this notebook we will look at ways for loading and analyzing data out-of-core using a batch processing method. Specifically, we will be using the split-apply-combine approach with a few different APIs. In R, we will use:
- `bigmemory` for memory mapping the variables: https://cran.r-project.org/web/packages/bigmemory/index.html 
- `bigtabulate`
- `biganalytics`
- `doMC`

In python, we will also be using a number of different libraries 
- `pandas`
- `numpy`
- `graphlab-create`

___
# 1.0 Downloading the dataset (using python)
You can access descriptions of the data files from here: http://stat-computing.org/dataexpo/2009/the-data.html. You can manually download each of the zipped files OR run the following script to download the files into a folder in the same directory as this notebook called "Data".

In the following blocks of code, we download the data and then decompress the files into .csv files. Each csv contains data for one year of airline flights. 


Note: If you want an alternative R version for performing the download operations, see:
- http://www.cybaea.net/journal/2010/08/05/Big-data-for-R/

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import sys
import time
import cPickle as pickle

#Here I needed to register (https://turi.com/register) and upgrade my graphlab using a free, academic-use-only product key:
# pip install --upgrade --trusted-host pypi.python.org graphlab-create 
# pip install --upgrade --no-cache-dir https://get.graphlab.com/GraphLab-Create/2.1/your registered email address here/your product key here/GraphLab-Create-License.tar.gz
import graphlab as gl
#gl.get_dependencies() #Just need to run this the first time if graphlab loads but a dependcy error ACTION REQUIRED note appears

# create a Data directory if not done already
path = "Airline/"
if not os.path.exists(path):
    os.mkdir( path, 0755 )
print(path)

In [ ]:
import urllib # this is part of the standard library for python
import bz2 # this is also part of the python standard library
#from urllib.request import urlretrieve
i = 0
for dirpath, dirnames, files in os.walk(path):
    if files and i==0:
        print("Files already downloaded and decompressed")
    if not files and i==0:
        years_to_download = range(1987,2009) # get the years 1987 through 2008
        baseurl = 'http://stat-computing.org/dataexpo/2009/%d.csv.bz2' 

        files = []
        for year in years_to_download:
            # prepare strings
            url_of_data_file = baseurl%(year) # get the URL for the data file
            save_as_filename = path + '%d.csv.bz2'%(year) 
            # save as this
            files += [save_as_filename] # save name of the compressed file

            # download file
            print('Downloading %s to %s' % (url_of_data_file, save_as_filename)) # progress update
            urllib.urlretrieve(url_of_data_file, save_as_filename) #execute download
            
        # Now lets decompress all the files
        for filename in files:#[path+'2006.csv.bz2',path+'2007.csv.bz2',path+'2008.csv.bz2']:
            # get file names
            filepath = filename
            newfilepath = filename[:-4]
            print('Decompressing', filepath,'to', newfilepath)

            # go through the decompressed chunks and write out to a decompressed file
            with open(newfilepath, 'wb') as new_file, bz2.BZ2File(filepath, 'rb') as file:
                for data in iter(lambda : file.read(100 * 1024), b''):
                    new_file.write(data)
    i = i+1

### If you want to delete the compressed files
Run the following block ONLY if you want to delets the compressed bz2 files from your system. 

In [ ]:
#!rm '/Users/darrenho/MSDSairline/'*.bz2

___
# 2.0 Loading data into memory
Now that the data has downloaded and been decompressed, we can load a single file into memory to ensure that everything decompressed correctly. For each file, we could load it into memory and then save the length of the file. Let's do this in both python and in R for two files (to keep runtime at a minimum). 

In [ ]:
# Loading individual files in python
import pandas as pd
import numpy as np
import sys

total_length = 0

for year in [1987,1988]:
    # get file name of the csv
    # note that we can also load in the raw .bz2 file in python (or R) 
    # but the decompression step for these files sizes takes a huge performance hit
    csvfile = path+'/%d.csv'%(year)
    print('loading', csvfile)
    sys.stdout.flush()
    
    # read the file and increment the lines count
    %time df = pd.read_csv(csvfile) # note that this is a big operation, especially since we just want the length
    # one way of making this shorter is to filter which columns we are loading
    
    total_length += len(df)

print('Answer from python:', total_length)
df.head()

___
So now we can see that there were about 6.5M different entries in the years 1987 and 1988. However, we really want to be working with one really large descriptor of all years, rather than loading up each year from disc each time. Actually, the operations we had above are much more appropriate for a SQL query, but they are not quite as scalable for some of the other analyses that we want to do later on. 

The python code is noticeably faster at loading in the data, so let's use python to pre-process the csv files and make sure all the data is an integer. All the text data must be coded as numeric (an integer) when we use R's big memory package, so lets go file by file and make sure its ready to use with R.


___

## 4.2 Preprocessing the CSV data in Python with Graphlab
So great, we can load up the same file as R did. It still took forever to preprocess that file and get it ready. So, we really want to answer this question:
**Is there an easier way to preprocess and concatenate all the files into one memory map using SFrames?** 

SFrames are more forgiving in terms of the data types that they can hold. Moreover, they load much more quickly than the parser used by `pandas`. Maybe we should try to read in each of the original CSV files as SFrames and concatenate them together into one SFrame. Would this be a quicker way to create the memory mapped file than all the preprocessing we did above?

The answer is, in fact, **yes**. It is much faster and the code is much easier to understand than what we performed earlier. And because `graphlab` supports more than one data type, we do not need to preprocess any of the data. Plus, when we write out to file, it happens more quickly and in a compressed version (on disk it will be compressed to about 2GB). 

Here is the code to perform all the concatenation we did earlier. I am also supplying graphlab with the data type for each column to ensure that the data is consistent. However, if I did not, Graphlab would try and guess the data type based on the first 100 lines of the csv file.

In [ ]:
if not os.path.exists(path+'airline_data_sframe_directory'):
    os.mkdir(path+'airline_data_sframe_directory', 0755)
    print('airline_data_sframe_directory directory created')
    
for dirpath, dirnames, files in os.walk(path+'airline_data_sframe_directory'):
    if files:
        print("Binary compressed files already exist")
        sf = gl.load_sframe(path+'airline_data_sframe_directory')
    if not files:
        # What about just loading up all the data using the SFrame Utility for loading CSV files?
        # We will need to make sure that the SFrame has consistent datatypes, so we will give the value for each header
        column_hints=[int,int,int,int,int,int,int,int,str,int,str,int,int,int,int,int,str,str,int,int,int,int,str,int,int,int,int,int,int]

        t = time.time()
        # now load the first SFrame
        sf = gl.SFrame() #.read_csv('Data/1987.csv',column_type_hints=column_hints)

        # and then append each SFrame in a for loop
        for year in range(1987,2009):
            print 'read %d lines, reading next file %d.csv'%(sf.shape[0],year)
            sys.stdout.flush()
            sftmp = gl.SFrame.read_csv('Airline/%d.csv'%(year),column_type_hints=column_hints)
            sf = sf.append(sftmp)

        print 'It took %.2f seconds to concatenate the memory mapped file'%(time.time()-t)

        t = time.time()
        print 'Saving...',
        sf.save(path+'airline_data_sframe_directory') # save a compressed version of this SFrame
        print 'took %.2f seconds'%(time.time()-t),'Shape of SFrame is',sf.shape

So yeah, we concatenated and loaded the file in ~320 seconds and saved a compressed binary version in about 1 minute (on my machine). That's quite a speedup. 

But can we perform operations upon it? Let's check how versatile these data structures are. Let's first try to replicate the functionality of finding the most popular airports to fly out of, like we did with `bigmemory`. 

In [ ]:
# CONVENIENCE BLOCK FOR THOSE LOADING FROM DISK HERE

# If you have already run the notebook above and just want to load up the data
# then you can reload the SFrame here
#import graphlab as gl
#
#sf = gl.load_sframe('Airline/airline_data_sframe_directory')

## 4.3 Analyzing popular Airports to fly from in Python
We now need to perform some operations using the split-apply-combine technique. However, in graphlab this all happens via one syntax (the `groupby` function). Let's see how it works. 

In [ ]:
# to perform grouping and splitting
# we need to specify (1) which column(s) to group the SFrame using, and 
#                    (2) what function we want to perform on the group
# in graphlab, we only have a few options for performing on each of the groups. 
# Here, lets keep it simple--let's group by the airport origin and then
#  use the builtin 'count' function to aggregate the results
# The result is another SFrame with the Unique origin names as a column and the
#  number of entries in each group in another column
%time sf_counts = sf.groupby('Origin', {'num_flights':gl.aggregate.COUNT()})
sf_counts

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

%matplotlib inline
plt.style.use('ggplot')

# As seen above, the sf_counts SFrame has the origin of the flight on the left
# and the count of flights on the right 

# let's grab the top 10 entries
sf_top = sf_counts.topk('num_flights',10) # this is builtin command in graphlab

airports = np.array(sf_top['Origin'])
counts = np.array(sf_top['num_flights'])

fig = plt.figure(figsize=(8,4))
plt.barh(range(len(counts)),counts)

# and set them on the plot
plt.yticks(range(len(airports)), airports)

plt.show()

___

## 4.4 Analyzing Depature Delays at Specific Times of the Day in Python
Now, let's try to do something a little more interesting, such as trying to perform the same operations that are in the Nolan text. Specifically, let's try to find the percentiles for late flights based upon the hour of the day that they depart. To do this we will:
- Create a new column for the hour of the day that a plane departed.
- Fix the `hours==24` and `hours==0` to be consistent
- Group by the hours of departure 
- Take percentiles of each group to see which ones are the latest

In [ ]:
from math import floor
# first, let's create a new column in this SFrame that has the departure time floored to the nearest hour
sf['DepTimeByHour'] = sf['CRSDepTime'].apply(lambda x: floor(x/100),dtype=int)
sf['DepTimeByHour'] # and print a few of them (note: the column has not been evaluated yet)

In [ ]:
# Let's now change the hours of the day that are equal to 24
# we need to be careful here becasue each column is not immutable
# in pandas this would be:
#    df.DepTimeByHour[df.DepTimeByHour==24] = 0
# but we cant just change a few values in the column, we need to change them all and replace them
# don't worry though, Graphlab does this smartly
sf['DepTimeByHour'] = sf['DepTimeByHour'].apply(lambda x: 0 if x==24 else x)
# again, this column has not been evaluated yet because that value has not yet been accessed

In [ ]:
# now lets group the SFrame by the hours and calculate the percentiles of each group
# here is where the lazy evaluation actually happens so this takes a little while to compute

# the groupby function will partition our SFrame into groups based upon the given column of data
# next, we need to tell graphlab what operations to perform on the group and what rows
# to do that, we send in a dictionary of names and 'operations' 
# We did a similar operation above with the 'COUNT' aggregator
# there are only a certain number of operators we can choose from, we will choose to use the 'QUANTILE'
#   aggregator on the column 'DepDelay'. We want to take the percentiles [0.90,0.99,0.999,0.9999]
# We can also perform other operations by adding entries in the dictonary
# So we will also take the 'MAX' of each group
import time
t = time.time()
delay_sf = sf.groupby('DepTimeByHour', 
                            {'delay_quantiles':gl.aggregate.QUANTILE('DepDelay', [0.90,0.99,0.999,0.9999]),
                             'delay_max':gl.aggregate.MAX('DepDelay')})
# this returns a new SFrame with the specified columns from each aggregation

print 'Took %.2f seconds to run'%(time.time()-t)

In [ ]:
# sort it by when departed and display it
delay_sf = delay_sf.sort('DepTimeByHour')
delay_sf

In [ ]:
# to use matplotlib, we need to convert over to numpy arrays
# this is a fine operation because the new aggregated SFrame we are 
# working (delay_sf) with is quite small
x = np.array(delay_sf['DepTimeByHour'])
y = np.array(delay_sf['delay_quantiles'])

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(x,y)
plt.ylabel('Minutes Delayed')
plt.xlabel('Hour of Day')


plt.subplot(1,2,2)
plt.plot(x,y)
plt.xlabel('Hour of Day')
plt.ylim(0,1400) # make the same axes as in the book
plt.legend(['0.9','0.99','0.999','0.9999'])

plt.show()

Oh no! This doesn't look like the graph from the book at all!! Actually, there is a really great reason for that...

Let's investigate further. The bottom two lines (percentiles 0.90 and 0.99) look very similar to what your book had, but the other two percentile values (0.999 and 0.9999) seemingly did not evaluate properly. 

Find out why by looking at the documentation for the quartile aggregator.
- https://dato.com/products/create/docs/graphlab.data_structures.aggregation.html#module-graphlab.aggregate
- Hint: there is a reason that the quantile calculation was really, really, really fast.
___

## 4.5 Calculate Plane Age with Dato
So let's keep moving and try to calculate the plane's age like we did in R.

In [ ]:
# only use years where the tail number was recorded
# we can manipulate the SFrame fairly easily in graphlab, so let's do it
sf_tmp = sf[['TailNum','Year','Month','DepDelay']][sf['Year']>1994]

In [ ]:
# lets try to make a function for getting the age of the plane
# First lets just save the plane's age in years
sf_tmp['FlightAge'] = 12*sf_tmp['Year']+sf_tmp['Month']-1

# and take the minimum of that in order to get its first flight
t = time.time()
sf_min_ages = sf_tmp[['TailNum','FlightAge']].groupby('TailNum',{'FirstFlight':gl.aggregate.MIN('FlightAge')})
print 'Took %.2f seconds to run'%(time.time()-t)

In [ ]:
# Now transform the FirstFlight Column into the original dataframe size
# to do that we can just do a join on a few columns of our sf
# this will save the flight age and the minimum in a new SFrame
%time sf_fewcols = sf_tmp[['TailNum','FlightAge']].join(sf_min_ages,on='TailNum',how='left') # long operation

In [ ]:
# and now we can simply subtract the new calculated quantity and add to the original SFrame
sf_tmp['Age'] = sf_fewcols['FlightAge']-sf_fewcols['FirstFlight']

## 4.6 Calculate a linear model from massive data with Python
Graphlab (much like `biganalytics`) will perform mini-batch linear regression and it will do it fast. Let's see what kind of output we get.

In [ ]:
# now look at the age and delay time in terms of regression (like your book)
%time lin_model = gl.linear_regression.create(sf_tmp['DepDelay','Age'].dropna(), target='DepDelay', features=['Age'])

In [ ]:
lin_model['coefficients']

Which again, gives us a positive association of Age and departure delay time (but could be very weak because we model nothing else). The value is of the same magnitude as what we found using R. It is slightly different because we need to use gradient descent (not batch methods, which are deterministic). The number of iterations and starting point for gradient methods will slightly (or not so slightly sometimes) affect the final coefficient values. 

___
# 5.0 Conclusions
Now that we have seen SFrames and bigmemory, which one should you choose? **As always, it depends.** I would recommend performing preprocessing with SFrames, almost always. They are much easier to work with. But their use of split-apply-combine is still lacking behind the abilities of R's bigmemory. Some operations are optimized, and if you can get away with only using those operations, then try just using SFrames. Since SFrames are great for adding and subtracting columns, that is another way to play around with the data before trying to use R. 

However, if you need a more flexible solution, bigmemory will need to be what you go with. And it is a really wonderful program... That's especially true when the aggregation function you use needs to do something rather complex. If you are running windows... SFrames are your goto. But, really, why are you running windows? Oh right, for SAS... Time to install a linux partition maybe... Or buy a mac and run bootcamp... You get the idea.

Another thing to keep in mind: Graphlab is actively being developed (and most of it is open source, like SFrames). It will one day have an R interface. It will probably get custom aggregation functions one day soon. 

The solutions I have provided here are just examples. Large datasets can be handled using databases really well (but eventually you run into limitations). 

And, as always, happy analyzing!

___

*This Notebook was created by Professor Eric Larson and modified by Darren Homrighausen

## References
[1] D. Lang and D. Nolan, Data Science in R: A Case Studies Approach to Computation Reasoning and Problem Solving. New York, New York: CRC Press.

[2] H. Wickham. Airline on-time performance Web page. http://statcomputing.org/dataexpo/2009/, 2009.

[3]  M. Kane and J. Emerson. biganalytics : A library of utilities for big.matrix objects of package bigmemory. http://cran.r-project.org/package=biganalytics , 2010. R package version 1.0.14.

[4]  M. Kane and J. Emerson. bigmemory : Manage massive matrices with shared memory and memory-mapped files. http://cran.r-project.org/package=bigmemory , 2011. R package version 4.2.11.
